<a href="https://colab.research.google.com/github/pedro00dk/ufpe-security-analysis/blob/master/ufpe_sa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UFPE Security Analysis

Fetching github repo to get database file.

In [1]:
!rm --recursive --force -- ./ufpe-security-analysis/
!git clone -- https://github.com/pedro00dk/ufpe-security-analysis
!ls -- ./ufpe-security-analysis/

Cloning into 'ufpe-security-analysis'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 42 (delta 20), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (42/42), done.
 README.md   ufpe_sa.ipynb  'UFPE SSI.xlsx'


#### Packages and imports

Install necessary packages for loading excel

In [2]:
!pip install xlrd

    100% |████████████████████████████████| 112kB 2.6MB/s 


In [3]:
import nltk
nltk.download('punkt')
nltk.download('mac_morpho')
import numpy as np
import pandas as pd
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package mac_morpho to /root/nltk_data...
[nltk_data]   Unzipping corpora/mac_morpho.zip.


### Dataframe Columns and Data

The database consists of an excel file with sheets per year of information. All of these sheets have to be loaded and merged. I printed only two lines of the dataframes to check the data formats and column names for concatenation.

In [4]:
xls = pd.ExcelFile('./ufpe-security-analysis/UFPE SSI.xlsx')
dfs = xls.parse(sheet_name=xls.sheet_names)

[display(y, df.sample(2)) for y, df in dfs.items()]

'2008'

,Data,Quadrante,Local,Horário,Tipo,Descrição Sumária,Partes,Medidas
128,16.05.08,NaN,Ñinf,19h30,colisão,Ocorrência 643-um FIAT UNO colidiu com o MONSA...,Jusyane Maria Nascimento de Lima/monsa,Prestou queixa a uma viatura.
110,29.04.08,Gama,CCSA,22H30,Registro de ocorrência,Ocorrência 609-gato que estava preso em uma ár...,LELIS/Cabo,Ñinf


'2009'

,Data,Quadrante,Local,Horário,Tipo,Descrição Sumária,Partes,Medidas
25,28.01.09,Beta,Editora universitária,10h15,Prestação de socorro,Ocorrência 1135 - 01 indivíduo se sentiu mal.,Srº Roberto Virgílio do Nascimento.,O GTO foi acionado.
78,14.03.09,NaN,Ñ inf,Ñ inf,Roubo,Ocorrência 2024 - 01 estudante foi vítima de u...,Ñ inf,Ñ inf


'2011'

...

'2014'

,Data,Quadrante,Local,horário,Tipo,Descrição Sumária,Partes,Medidas
38,2014-02-06 00:00:00,Gama,CFCH,16h40,Registro de ocorrência,Ocorrência 7251-segundo registro um aluna foi ...,Aluna Mariana Cavalcante Albuquerque.,O GTO foi ao local mas os elementos já haviam ...
164,2014-05-15 00:00:00,Gama,Laguinho próximo a grade do CAP,14h25,Tentativa de assalto,Ocorrência 7547 - segundo registro duas menina...,Ñ inf,O GTO acionou a cirene da moto e os elementos ...


'2015'

,Data,Quadrante,Local,horário,Tipo,Descrição Sumária,Partes,Medidas,Unnamed: 8,Unnamed: 9,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35
256,2015-05-30,Delta,Restaurante do HC,09h00,Registro de Ocorrência,"Ocorrência 303 TURMA ""C"" - segundo registro os...",Aluna: Ilgiza Iangildina (Curso de Letras).,A aluna foi conduzida até a casa feminina da U...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
289,2015-06-19,Gama,Restaurante Universitário,12h40,Prestação de Socorro,"Ocorrência 0345 TURMA ""C"" - segundo registro u...",Acompanhante: Jeferson Ramos (Curso de Engenha...,"O estudante, juntamente com o seu acompanhante...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


[None, None, None, None, None, None, None]

### Dataframes Concatenation

Processing the dates of all dataframes before merging because they have different formats, it will help to maitain the correctness of the processed dates. Some cleaning in the columns names is also necessary to make then match in concatenation.

In [5]:
# lower all columns and remove extra spaces
dfs = {y: df.rename(lambda c: c.lower().strip(), axis='columns') for y, df in dfs.items()}

# drop columns
drop_rule = re.compile(r'(?x) descrição \s sumária | medidas | unnamed.*')
dfs = {y: df.drop([c for c in df.columns if drop_rule.match(c)], axis='columns') for y, df in dfs.items()}

# unify null elements
na_rule = re.compile(r'(?i)ñ.*')
dfs = {y: df.replace(na_rule.pattern, np.nan, regex=True) for y, df in dfs.items()}

# unify date columns, date have different formats along all dataframes, so they have to be processed before concat
date_format = {
    '2008': '%d.%m.%y',
    '2009': '%d.%m.%y',
    '2011': '%d.%m.%y',
    '2012': '%d.%m.%y',
    '2013': '%Y-%m-%d',
    '2014': '%Y-%m-%d', #'%Y-%m-%d %h:%m:$s'
    '2015': '%Y-%m-%d'
}
dfs = {
    y: df.assign(**{'data': lambda df: pd.to_datetime(df['data'], errors='coerce', format=date_format[y], exact=False)})
    for y, df in dfs.items()
}

# merge dataframes
df = pd.concat(dfs, ignore_index=True)
df.sample(10)

,data,quadrante,local,horário,tipo,partes
2026,2015-08-04,NaN,NaN,18h30,Roubo ZLI,Estudante: Ana Paula dos Santos Raimundo.
1820,2015-03-23,NaN,NaN,11h40,Furto Privado,Srº: Rafael Rodrigues.
876,2013-01-26,Gama,CFCH,14h10,Prestação de socorro,Davi Filipe
66,2008-02-27,ZLI,Campus,NaN,Registro de ocorrência,NaN
1523,2014-06-10,Beta,Departamento de Física,17h00,Registro de ocorrência,NaN
1788,2015-03-03,Gama,Farmácia,13h30,Prestação de Socorro,Senhora: Cristiane dos Santos.
1958,2015-06-01,Alfa,Gradil da Reitoria,19h25,Registro de Ocorrência,Senhora: Cristiane Alves do Nascimento (Funcio...
126,2008-05-14,NaN,NaN,16h30,Tentativa de roubo,NaN
387,2011-01-29,Delta,Biotério,NaN,Arrombamento de Instalações,Larisse Cardoso
257,2009-02-17,Beta,Oceanografia,20h00,Registro de ocorrência,Srº Yuri da construtora Aragão.


### Process DataFrame columns

In [6]:
pdf = df.copy()

pdf['horário'] = pd.to_datetime(df['horário'], errors='coerce', format='%Hh%M', exact=False)

pdf['quadrante'] = df['quadrante']\
    .fillna('')\
    .apply(lambda e: str(e).strip().lower())\
    .apply(lambda e: 'None' if len(e) <= 1 else e)\
    .apply(lambda e: 'externa' if 'externa' in e else e)\
    .apply(lambda e: 'externa' if e.startswith('z') and e.endswith('e') else e)\
    .apply(lambda e: 'interna' if e.startswith('z') else e)\
    .apply(lambda e: e[:e.index('/')] if '/' in e else e)

places_rules = [
    ('CAC-CFCH', re.compile(r'(?ix) cfch | cac | (a|á)rtes | gere | sintuf')),
    ('CE',       re.compile(r'(?ix) ((^|\s) ce ($|\s)) | col(é|e)gio | aplica | educaç | ((^|\s)cap($|\s))')),
    ('CCSA',     re.compile(r'(?ix) ccsa | cssa | adu(e?)fpe | economia | lamsa | ((^|\s)de(q|c)($|\s)) | hotel | lac | turism | qu(í|i)mic')),
    ('CTG',      re.compile(r'(?ix) ctg | oce(o|a)no | museu | fae | eng. | engenh')),
    ('CCEN',     re.compile(r'(?ix) ccen | dep .* físi | matem')),
    ('CCB',      re.compile(r'(?ix) ccb')),
    ('CCS',      re.compile(r'(?ix) ccs | odonto | dermato | medic | fono | histono | f(á|a)rm(á|a)c | nutri | anato | faci | biot | buco | psico | svo | sa(ú|u)de | antib | nasc')),
    ('CIN',      re.compile(r'(?ix) cin | inform')),
    ('AREA2',    re.compile(r'(?ix) ((á|a)rea \s? (2|ii)) | pcu | prefeit | den')),
    ('RU',       re.compile(r'(?ix) ru | restaurante')),
    ('NTI',      re.compile(r'(?ix) ((^|\s) nti) | n(ú|u)cleo de tecn')),
    ('NEDF',     re.compile(r'(?ix) nedf | nefd | dine | nai | fisio | terapi | (educ .* ica) | arqueo | n(ú|u)cleo | quadra')),
    ('LANSA',    re.compile(r'(?ix) lansa')),
    ('CECINE',   re.compile(r'(?ix) cecine')),
    ('FARMACIA', re.compile(r'(?ix) f(á|a)rm(á|a)cia')),
    ('CEU',      re.compile(r'(?ix) ((^|\s) ceu ($|\s|-)) | masc | femin')),
    ('DGOS',     re.compile(r'(?ix) dgos')),
    ('REITORIA', re.compile(r'(?ix) reitoria | proa | caixa | brasi | ((^|\s)nae($|\s|\.)) | proacad')),
    ('BR101',    re.compile(r'(?ix) (br .* 101?) | sudene | viadut | girad')),
    ('HC',       re.compile(r'(?ix) hc | hospital | cl(í|i)nicas')),
    ('ENTRADA',  re.compile(r'(?ix) entrada | sa(í|i)da | editora | (o|ô)nibus | guarita | ((^|\s)cos($|\s))')),
    ('AVENIDA',  re.compile(r'(?ix) avenida | av\. | ceco(n|m) | conven | central | arqui | reitore | teatr | ((^|\s)bc($|\s))')),
    ('CONCHA',   re.compile(r'(?ix) concha')),
    ('COOPER',   re.compile(r'(?ix) cooper')),
    ('CAST_AGUA',re.compile(r'(?ix) castelo | agua')),
    ('CASA-ESTD',re.compile(r'(?ix) casa | mist')),
    ('CLUBE',    re.compile(r'(?ix) clube')),
    ('LAGO',     re.compile(r'(?ix) lago | lagu | riach | cavouc')),
    ('CAMPUS',   re.compile(r'(?ix) campus | ufpe')),
    ('TVU',      re.compile(r'(?ix) tvu | ((^|\s)tv($|\s)) | tv \s u | r(a|á)dio')),
    ('none',     re.compile(r'(?ix) none')),
    ('OUTROS',   re.compile(r'(?ix) .*')),
]
place_matcher = lambda e: [place for place, rule in places_rules if rule.search(e)][0]
pdf['local'] = df['local']\
    .fillna('None')\
    .apply(lambda e: str(e).strip().lower())\
    .apply(place_matcher)

type_rules = [
    ('HOMICIDIO',re.compile(r'(?ix) homic(í|i)')),
    ('SUICIDIO', re.compile(r'(?ix) suic(í|i)')),
    ('ESTUPRO',  re.compile(r'(?ix) estupr | estrup')),
    ('AGRESSÃO', re.compile(r'(?ix) agress')),
    ('ROUBO',    re.compile(r'(?ix) assal | arrast |roub')),
    ('FURTO',    re.compile(r'(?ix) furt')),
    ('BRIGA',    re.compile(r'(?ix) brig')),
    ('ARROMBAM.',re.compile(r'(?ix) arromb | inva(s|d)')),
    ('APREENCAO',re.compile(r'(?ix) apreen | arma')),
    ('DROGAS',   re.compile(r'(?ix) Drog | entorp')),
    ('SUSPEITOS',re.compile(r'(?ix) suspeit | abordag')),
    ('COLISÃO',  re.compile(r'(?ix) colis | aciden')),
    ('SOCORRO',  re.compile(r'(?ix) presta | socorr')),
    ('INCENDIO', re.compile(r'(?ix) inc(ê|e)nd')),
    ('CARRO VUL',re.compile(r'(?ix) carr | veic')),
    ('DANO_PATR',re.compile(r'(?ix) patrim | dano')),
    ('ANIMAL',   re.compile(r'(?ix) anima')),
    ('BO',       re.compile(r'(?ix) regist | ocorr | bolet')),
    ('none',     re.compile(r'(?ix) none')),
    ('OUTROS',   re.compile(r'(?ix) .*')),
]
type_matcher = lambda e: [type for type, rule in type_rules if rule.search(e)][0]

pdf['tipo'] = df['tipo']\
    .fillna('None')\
    .apply(lambda e: str(e).strip().lower())\
    .apply(type_matcher)

display(
    pdf['local'].value_counts().head(8),
    pdf['tipo'].value_counts().head(8),
)

pdf.sample(10)

none        239
CCS         211
CAC-CFCH    209
NEDF        155
CCSA        140
CTG         111
AVENIDA     111
REITORIA    108
Name: local, dtype: int64

BO           751
ROUBO        335
SOCORRO      224
FURTO        194
COLISÃO      163
ARROMBAM.    131
CARRO VUL     92
APREENCAO     54
Name: tipo, dtype: int64

,data,quadrante,local,horário,tipo,partes
832,2013-01-06,delta,COOPER,1900-01-01 08:20:00,ARROMBAM.,Elias Pereira (informante)
1355,2014-02-11,delta,CCS,1900-01-01 13:15:00,BO,Aluna Iane Silveira Melo e Orientador Gilberto...
586,2012-01-18,delta,HC,NaT,SOCORRO,Hugo paciente
1606,2014-08-17,beta,CIN,1900-01-01 21:30:00,INCENDIO,Aluno: Diogo Medeiro de Almeida / Gerente de i...
855,2013-01-19,None,none,1900-01-01 19:15:00,SOCORRO,Rubens Augusto da Silva
1247,2013-10-21,alfa,CLUBE,1900-01-01 18:10:00,BO,Jose Roberto(segurança UFPE)
899,2013-02-07,gama,CE,NaT,BO,NaN
1143,2013-07-08,delta,HC,1900-01-01 11:25:00,BO,NaN
1517,2014-06-08,alfa,NEDF,1900-01-01 11:48:00,SUSPEITOS,NaN
1715,2015-01-17,gama,CE,1900-01-01 00:20:00,ARROMBAM.,Informante: vigilante da TKS lotado no CFCH.


### Create a POS-Tagger

FOR TEST PURPOSES, I DONT KNOW IF IT WILL WORK!
This tagger will be used to get personal names from 'partes' column.

In [7]:
from nltk.corpus import mac_morpho

tsents = mac_morpho.tagged_sents()
simplify_tag = lambda t: t[t.index("+")+1:] if '+' in t else t
tsents = [[(w.lower(), simplify_tag(t)) for (w,t) in sent] for sent in tsents if sent]

train = tsents[:int(len(tsents) * (2 / 3))]
test = tsents[int(len(tsents) * (2 / 3)) + 1:]

default_tagger = nltk.DefaultTagger('n') # noums have higher frequency
print('default tagger accuracy:', default_tagger.evaluate(test))

unigram_tagger = nltk.UnigramTagger(train, backoff=default_tagger)
print('unigram tagger accuracy:', unigram_tagger.evaluate(test))

bigram_tagger = nltk.BigramTagger(train, backoff=unigram_tagger)
print('bigram tagger accuracy:', bigram_tagger.evaluate(test))

default tagger accuracy: 0.0
unigram tagger accuracy: 0.7690193016457291
bigram tagger accuracy: 0.7848793705237425


In [8]:
bigram_tagger.tag(nltk.word_tokenize('Proprietária do Kia: Adriana Maria de Abreu'))

[('Proprietária', 'n'),
 ('do', 'NPROP'),
 ('Kia', 'n'),
 (':', ':'),
 ('Adriana', 'n'),
 ('Maria', 'n'),
 ('de', 'PREP'),
 ('Abreu', 'n')]

In [18]:
pdf['tipo'][pdf['quadrante'] == 'alfa'].value_counts()

BO           186
ROUBO         82
SOCORRO       45
COLISÃO       41
ARROMBAM.     38
FURTO         37
APREENCAO     16
SUSPEITOS     11
DANO_PATR     11
CARRO VUL     10
INCENDIO       8
AGRESSÃO       6
DROGAS         4
ANIMAL         2
OUTROS         2
BRIGA          2
HOMICIDIO      1
Name: tipo, dtype: int64

### Testas a serem usados

* Teste de confiaça para as médias de ROUBOS por quadrante por ano
* Teste de Freeman
* Teste de Nemenyi
* Series tempoorais (talvez?)